# Домашнее задание Урока 3

### Тема "Коллаборативная фильтрация."

In [1]:
import time
import numpy as np
import pandas as pd

from implicit.als import AlternatingLeastSquares

from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

from IPython.display import display

# lib_rs == мои скрипты для Reccomendation System
from lib_rs import data_to_sparse_matrix, precision_at_k_
from lib_rs import tfidf_weight_, bm25_weight_, cosine_weight_
from lib_rs import get_recommendations

## Теоретическая часть

### Задание 1
- Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи?

Реализаций User-User алгоритма не существует (например в implicit есть ItemItemRecommender, но нет UserUserRecommender). User-User и Item-Item по сути одинаковы и отличаются только входными данными. 

- Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?

Да, если транспонировать interaction_matrix.

- В чем принципиальные отличия item-item рекомендаций от ALS?
  - ALS в отличие от item-item использует итератвный алгоритм с колличеством итераций (iterations) и скоростью обучения (regularization) для подбора коэффициентов которые в том числе зависят от колличества факторов (factors).

### Задание 2
- Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров
  - Есть 3 параметра на пересечении user_id и item_id - quantity, sales_value.
  - Кроме sales_value nакже есть retail_disc и coupon_match_disc.


- Возможные варианты:
  1. Можно строить "таргетирование" веса, например по топ-5000 по колличеству продаж или по выручке (сумме покупок).
  2. По выручке с учетом дискаунтов (sales_value - retail_disc и coupon_match_disc).
  3. Веса учитывающие колличество продаж или выручки с учетом K ближайших пользователей или товаров.
  4. Можно сделать веса учитывающие популярность брендов или типов товаров.

### Задание 3
- Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)
   - Увеличение числа факторов (factors) или числа итераций (iterations) значитильно увеличиват время обучения модели.
   - Можель воспринимает только sparse матрицы (матрицы с разрежением данных), данные должны быть типа float или double.
   - ALS это линейная модель с точки зрения факторов (фичей).

### Задание 4
- Мы рассматривали bm_25_weight. Опишите, как он работает.

Ответ:
- Формулы BM25 описаны в Интернете, например в википедии.
- BM25 ранжирует вероятности выбора пользователями товаров и в зависимости от частоты выбора продукта всеми пользователями и в зависимости от того, какие продукты выбрал конкретный пользователь из всего набора продуктов.

In [2]:
item_names = ['Молоко', 'Йогурт', 'Мясо', 'Рыба']
user_names = ['Мария', 'Анна', 'Глеб', 'Никита', 'Ёси']
users_items = np.array([[1,1,1,1],[1,1,0,0],[0,1,1,1],[1,0,1,0], [0,1,0,1]])
df = pd.DataFrame(data=users_items, index=user_names, columns=item_names)
df

,Молоко,Йогурт,Мясо,Рыба
Мария,1,1,1,1
Анна,1,1,0,0
Глеб,0,1,1,1
Никита,1,0,1,0
Ёси,0,1,0,1


In [3]:
data_train = pd.DataFrame(users_items, index=range(len(user_names)), columns=range(len(item_names)))
cols = data_train.columns
data_train['user_id'] = data_train.index
data_train = pd.melt(data_train, id_vars=['user_id'], value_vars=cols.values).\
                 rename(columns={'variable': 'item_id', 'value': 'quantity'}).reindex()
data_train.head()

,user_id,item_id,quantity
0,0,0,1
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0


In [4]:
sparse_user_item, id_to_itemid, userid_to_id = data_to_sparse_matrix(data_train)
sparse_user_item.toarray().T

array([[1., 1., 0., 1., 0.],
       [1., 1., 1., 0., 1.],
       [1., 0., 1., 1., 0.],
       [1., 0., 1., 0., 1.]])

In [5]:
bm25 = bm25_weight_(sparse_user_item.toarray().T).T.toarray()
bm25

array([[-0.23762164, -0.18865904, -0.23762164, -0.23762164],
       [ 0.30634758,  0.2432238 ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.30634758,  0.        ,  0.30634758,  0.        ],
       [ 0.        ,  0.2432238 ,  0.        ,  0.30634758]])

In [6]:
pd.DataFrame(bm25, index=user_names, columns=item_names)

,Молоко,Йогурт,Мясо,Рыба
Мария,-0.237622,-0.188659,-0.237622,-0.237622
Анна,0.306348,0.243224,0.000000,0.000000
Глеб,0.000000,0.000000,0.000000,0.000000
Никита,0.306348,0.000000,0.306348,0.000000
Ёси,0.000000,0.243224,0.000000,0.306348


- Как сделать рекомендации только на основе bm_25?
- (Можно и нужно пользоваться любыми источниками, приложите на них ссылки).

Для каждого пользователя выбрать топ К продуктов на остовании BM25.

In [7]:
K = 2
df1 = df.copy()
df1['Рекомендация'] = [list(np.array(item_names)[bm25[user_id].argsort()[::-1]][:K]) for user_id in range(len(df))]
df1

,Молоко,Йогурт,Мясо,Рыба,Рекомендация
Мария,1,1,1,1,"[Йогурт, Рыба]"
Анна,1,1,0,0,"[Молоко, Йогурт]"
Глеб,0,1,1,1,"[Рыба, Мясо]"
Никита,1,0,1,0,"[Мясо, Молоко]"
Ёси,0,1,0,1,"[Рыба, Йогурт]"


- Какие еще способы перевзвешивания user-item матрицы Вы знаете / можете предложить (ещё 2-3 способа)?
  - TFIDF, Cosine Similarity, Pearson Similarity, KNN.
  - Реализации алгоритмов которые нашел приведены в lib_rs [tfidf_weight, bm25_weight, cosine_weight].

In [8]:
algorithms = [['tfidf_weight', tfidf_weight_], ['bm25_weight', bm25_weight_], ['cosine_weight', cosine_weight_]]
for name, algorithm in algorithms:
    print(name)
    display(pd.DataFrame(algorithm(sparse_user_item.toarray().T).T.toarray(), index=user_names, columns=item_names))

tfidf_weight


,Молоко,Йогурт,Мясо,Рыба
Мария,-0.223144,-0.223144,-0.223144,-0.223144
Анна,0.287682,0.287682,0.000000,0.000000
Глеб,0.000000,0.000000,0.000000,0.000000
Никита,0.287682,0.000000,0.287682,0.000000
Ёси,0.000000,0.287682,0.000000,0.287682


bm25_weight


,Молоко,Йогурт,Мясо,Рыба
Мария,-0.237622,-0.188659,-0.237622,-0.237622
Анна,0.306348,0.243224,0.000000,0.000000
Глеб,0.000000,0.000000,0.000000,0.000000
Никита,0.306348,0.000000,0.306348,0.000000
Ёси,0.000000,0.243224,0.000000,0.306348


cosine_weight


,Молоко,Йогурт,Мясо,Рыба
Мария,0.57735,0.5,0.57735,0.57735
Анна,0.57735,0.5,0.00000,0.00000
Глеб,0.00000,0.5,0.57735,0.57735
Никита,0.57735,0.0,0.57735,0.00000
Ёси,0.00000,0.5,0.00000,0.57735


## Практическая часть


##### Загрузка данных<a class="anchor" id="load_data"></a>

In [9]:
data = pd.read_csv('./data/retail_train.csv')

##### Разделение на train и test

In [10]:
test_size_weeks = 3
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks].copy()
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks].copy()
data_train_copy, data_test_copy = data_train.copy(), data_test.copy()
data_train.shape[0], data_test.shape[0]
data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


##### Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [11]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result['actual'] = result['actual'].apply(lambda x: list(x))
result_copy = result.copy()
result.head(3)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."


##### Топ 5000 товаров

In [12]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

##### Заведем фиктивный item_id (если юзер НЕ покупал товары из топ-5000, то он "купил" такой товар)

In [13]:
data_train = data_train_copy.copy()
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

##### sparse матрица

In [14]:
sparse_user_item, id_to_itemid, userid_to_id = data_to_sparse_matrix(data_train)

In [15]:
algorithms = [['als', None], ['tfidf_weight', tfidf_weight_], ['bm25_weight', bm25_weight_], ['cosine_weight', cosine_weight_]]
models = []
for name, algorithm in algorithms:
    print(f"{name:20} :")
    sparse_user_item_ = sparse_user_item if algorithm is None else algorithm(sparse_user_item.toarray().T).T
    start = time.process_time()
    model = AlternatingLeastSquares(factors=100, regularization=0.001, iterations=15, calculate_training_loss=True, num_threads=4)
    model.fit(sparse_user_item_.T, show_progress=True)
    models.append(model)
    print(f"     fit: {time.process_time()-start:.3} sec")
    start = time.process_time()
    result[name] = result['user_id'].apply(lambda user_id: get_recommendations(model, sparse_user_item_,
                                                                               userid=user_id, N=5,
                                                                               id2i=id_to_itemid, u2id=userid_to_id))
    print(f"  result: {time.process_time()-start:.3} sec")

als                  :


  0%|          | 0/15 [00:00<?, ?it/s]

     fit: 13.1 sec
  result: 43.8 sec
tfidf_weight         :


  0%|          | 0/15 [00:00<?, ?it/s]

     fit: 14.7 sec
  result: 71.0 sec
bm25_weight          :


  0%|          | 0/15 [00:00<?, ?it/s]

     fit: 15.8 sec
  result: 85.6 sec
cosine_weight        :


  0%|          | 0/15 [00:00<?, ?it/s]

     fit: 14.4 sec
  result: 68.4 sec


In [16]:
data_test = data_test_copy.copy()
data_test.loc[~data_test['item_id'].isin(top_5000), 'item_id'] = 999999

In [17]:
sparse_user_item_test, id_to_itemid_, userid_to_id = data_to_sparse_matrix(data_test)

In [18]:
for i, (name, algorithm) in enumerate(algorithms):
    model = models[i]
    start = time.process_time()
    name = name + '_test'
    result[name] = result['user_id'].apply(lambda user_id: get_recommendations(model, sparse_user_item_test,
                                                                               userid=user_id, N=5,
                                                                               id2i=id_to_itemid, u2id=userid_to_id))
    print(f"  result: {time.process_time()-start:.3} sec")

  result: 7.75 sec
  result: 7.04 sec
  result: 6.5 sec
  result: 6.74 sec


### Подбор матрицы $c_{ui}$
- Попробуйте различные варианты матрицы весов (3+ вариантов).
- Обучите алгоритм для различных $C$. 
- В качестве результата приведите таблицу: матрица весов - результат на train и validation.
- Сделате качественные выводы.

In [19]:
result.head(2)

,user_id,actual,als,tfidf_weight,bm25_weight,cosine_weight,als_test,tfidf_weight_test,bm25_weight_test,cosine_weight_test
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[995242, 1033142, 979707, 1005186, 1082185]","[1051211, 1033142, 1005186, 979707, 1105488]","[999999, 1082185, 1135694, 963835, 8090541]","[1082185, 999999, 995242, 840361, 1029743]","[833025, 1071939, 995785, 1110572, 1046262]","[1046262, 970202, 1071939, 833025, 1022003]","[999999, 835788, 1090017, 995785, 968057]","[833025, 859075, 1007195, 995785, 962229]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1133018, 1106523, 910032, 1022003, 826249]","[951590, 910032, 1051069, 938700, 9337581]","[951590, 826249, 9337581, 9337369, 1044078]","[1098066, 826249, 999999, 1082185, 1044078]","[874736, 907631, 6034857, 962118, 1039589]","[907631, 874736, 962118, 940947, 882441]","[1066783, 1133877, 868909, 9526412, 9396770]","[874736, 1103105, 907631, 6463789, 906838]"


In [20]:
for x in result.columns:
    if x in ['user_id', 'actual'] or x.endswith('_test'):
        continue
    r_train = result.apply(lambda row: precision_at_k_(row[x], row['actual']), axis=1).mean()
    r_test = result.apply(lambda row: precision_at_k_(row[x+'_test'], row['actual']), axis=1).mean()
    print(f"{x:20} : {r_train:.5f} {r_test:.5f}")

als                  : 0.18658 0.04006
tfidf_weight         : 0.15289 0.02615
bm25_weight          : 0.14035 0.05583
cosine_weight        : 0.15769 0.03761


##### Выводы:
- Взвешивание BM25 и Cosine привело к улучшению метрики Presision@K на тест.
- Странно, но TFIDF привело к снижению метрики Presision@K на тест.
- Взвешивание снижает Presision@K на train, что скорее всего сигнализирует о переобученности ALS.

----
### ПРИМЕЧАНИЯ: 
- НЕ УСПЕЛ ДОДЕЛАЛАТЬ.
- ПОТРАТИЛ СЛИШКОМ МНОГО ВРЕМЕНИ НА ИНТЕРНЕТ И ОТЛАДКУ КОДА.
----

### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.GridSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

**P.S.** Не пишите отписки в качестве выводов. Мне интресены Ваши рассуждения, трудности, с которыми Вы сталкнулись и что-то, что Вас удивило. Если выводы контринтуитивны - напишите об этом, в этом нет ничего страшного!

----
### Ссылки:
- People meet recommender systems. Factorization https://habr.com/ru/post/486802/
- Collaborative Filtering for Implicit Feedback Datasets http://yifanhu.net/PUB/cf.pdf https://www.researchgate.net/publication/220765111_Collaborative_Filtering_for_Implicit_Feedback_Datasets
- Fast Matrix Factorization for Online Recommendation with Implicit Feedback https://www.comp.nus.edu.sg/~xiangnan/papers/sigir16-eals-cm.pdf
- BPR: Bayesian Personalized Ranking from Implicit Feedback https://arxiv.org/ftp/arxiv/papers/1205/1205.2618.pdf
- Explicit Matrix Factorization: ALS, SGD, and All That Jazz https://blog.insightdatascience.com/explicit-matrix-factorization-als-sgd-and-all-that-jazz-b00e4d9b21ea
- ALS Implicit Collaborative Filtering https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe
- Role of Matrix Factorization Model in Collaborative Filtering Algorithm: A Survey https://arxiv.org/ftp/arxiv/papers/1503/1503.07475.pdf
- Alternating Least Square for Implicit Dataset with code https://towardsdatascience.com/alternating-least-square-for-implicit-dataset-with-code-8e7999277f4b
- Okapi BM25 https://en.wikipedia.org/wiki/Okapi_BM25